In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import r2_score
import glob


## Introduction
This reproduces the naive solution from the [notebook](https://www.kaggle.com/jiashenliu/introduction-to-financial-concepts-and-data) provided by the contest organizers, but replaces data manipulation in pandas with a straight numpy calculation. For a 10x speed improvement

In [ ]:
# A function to calculate realized volatility for all time intervals in a single book file
def realized_volatility_single_stock(file_path, prediction_column_name):
    df_book_data = pd.read_parquet(file_path)
    stock_id = file_path.split('=')[1]
    time_ids, bpr, bsz, apr, asz = (df_book_data[col].values for col in ['time_id', 'bid_price1','bid_size1','ask_price1','ask_size1' ])
    wap = (bpr * asz +apr * bsz) / (asz + bsz)
    log_wap = np.log(wap)
    ids, index = np.unique(time_ids, return_index=True)

    splits = np.split(log_wap, index[1:])
    ret=[]
    for time_id, x in zip(ids.tolist(), splits):
        log_ret = np.diff(x)
        volatility = np.sqrt((log_ret ** 2).sum())
        ret.append((f'{stock_id}-{time_id}', volatility.item()))
    return pd.DataFrame(ret, columns=['row_id', prediction_column_name])

In [ ]:
def realized_volatility_all(files_list, prediction_column_name):
    return pd.concat( [realized_volatility_single_stock(file, prediction_column_name) for file in files_list])

## Run on the train set to sanity check

In [ ]:
list_order_book_file_train = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/*')

In [ ]:
%%time
df_past_realized_train = realized_volatility_all(list_order_book_file_train, 'pred')

In [ ]:
train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]
df_joined = train.merge(df_past_realized_train[['row_id','pred']], on = ['row_id'], how = 'left')

In [ ]:
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))
R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}')

# Submission

As a last step, we will make a submission via the tutorial notebook -- through a file written to output folder.  The naive submission scored a RMSPE 0.327 on public LB, the room of improvement is big for sure!

In [ ]:
list_order_book_file_test = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/*')
df_naive_pred_test = realized_volatility_all(list_order_book_file_test,'target')
df_naive_pred_test.to_csv('submission.csv',index = False)

In [ ]:
pd.read_csv('submission.csv').head()